In [1]:
import pandas as pd
import time

In [2]:
df = pd.read_csv("diabetes.csv")
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['Outcome'])
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
bst = XGBClassifier()
distributions  = {
    'max_depth': range(1, 21),
    'reg_alpha': range(0, 100, 10),
    'reg_lambda': range(0, 100, 10),
    'n_estimators': range(10, 250, 10)
}
clf = RandomizedSearchCV(bst, distributions, random_state=0, scoring='f1')
start_time = time.time()
search = clf.fit(X, y)
print("Гиперпараметры: ", search.best_params_)
print(f"F1: {search.best_score_}")
print(f"Затраченное время: {time.time() - start_time}")

Гиперпараметры:  {'reg_lambda': 30, 'reg_alpha': 0, 'n_estimators': 100, 'max_depth': 14}
F1:  0.6409234040812989
Затраченное время:  3.26948881149292


In [6]:
from sklearn.metrics import f1_score
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
def objective(params):
    model = XGBClassifier(**params)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    return {'loss': -f1_score(y_test, pred), 'status': STATUS_OK}

space = {'max_depth': hp.choice('max_depth', range(1, 21)),
         'reg_alpha': hp.choice('reg_alpha', range(0, 100, 10)),
         'reg_lambda':  hp.choice('reg_lambda', range(0, 100, 10)),
        'n_estimators': hp.choice('n_estimators', range(10, 250, 10))
        }
trials = Trials()
start_time = time.time()
print(f"Гиперпараметры: {fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=40, trials=trials)}")
print(f"F1: {-trials.best_trial['result']['loss']}")
print(f"Затраченное время: {time.time() - start_time}")

100%|██████████| 40/40 [00:02<00:00, 15.12trial/s, best loss: -0.7215189873417721]
Гиперпараметры: {'max_depth': np.int64(16), 'n_estimators': np.int64(16), 'reg_alpha': np.int64(0), 'reg_lambda': np.int64(3)}
F1: 0.7215189873417721
Затраченное время: 2.6529600620269775


Таким образом, за время меньше, чем при случайном поиске, мы получили F1 качество лучше, чем при ручном поиске, причём за меньшее время